In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from definitions import *

In [ ]:
R = 10000e-10 # m
z = np.linspace(0,7e-6,1000)
xi = z / R
G_xi = G(z, R)
G_xi_2 = G(z, 2 * R)

plt.plot(z*1e6,G_xi)
plt.plot(z*1e6,G_xi_2)
plt.xlabel('z [um]')
plt.ylabel('G(z)')
plt.grid()

In [ ]:
R = 10000e-10 # m
Rs = [50e-9, 100e-9, 200e-9]
for R in Rs:
    z = np.linspace(0,500e-9,1000)
    G_xi = G_norm(z, R)
    # G_xi_2 = G_norm(z, 2 * R)

    plt.plot(z*1e9,G_xi, label=rf'$R = {round(R * 1e9)}$ nm')
    # plt.plot(z*1e6,G_xi_2)
plt.xlabel(r'$\delta$ [nm]')
plt.ylabel(r'$G_0(\delta)$')
plt.xlim((0,500))
plt.grid()
plt.legend()
plt.savefig("docs/analytical-G0.eps",bbox_inches="tight", pad_inches=0, format='eps')

In [ ]:
P = P_analytical(z, R, t=1e-3, wavelength=2.165e-10)
plt.plot(z*1e6,P)
plt.xlabel('z [um]')
plt.ylabel('P(z)')
plt.grid()

In [ ]:
z_sim, P_sim = np.genfromtxt(f'simulation-fit.txt', delimiter=',',usecols=(0,1),unpack=True)

plt.plot(z*1e6,P, label='Analytical')
plt.plot(z_sim*1e6,P_sim,'.', label='Fitted to simulation')
plt.xlabel('z [um]')
plt.ylabel('P(z)')
plt.legend()
plt.grid()

# Analytical form factor
Dilute SANS samples have $Q$-ranges corresponding to their form factor

In [ ]:
def form_factor(Q, R):
    return (3 * (np.sin(Q * R) - Q * R * np.cos(Q * R))/ (Q * R) ** 3)**2

Rs = [50e-9, 300e-9, 2000e-9]
Q = np.linspace(0, 0.015e10,100000)
# for R in Rs:
R = 100e-9
ff = form_factor(Q,R)
plt.plot(Q * 1e-10,ff)
# plt.xscale('log')
plt.yscale('log')
plt.xlabel(r'$Q$ [Å$^{-1}$]')
plt.ylabel(r'$P(Q)$')
plt.xlim((0.0,0.015))

plt.ylim((1e-7,1.1))
plt.axvline(0.1 / R * 1e-10, linestyle='--', color='red', label=r'$Q_\text{min} = 0.0001$ Å$^{-1}$')
plt.axvline(10 / R * 1e-10, linestyle='--', color='green', label=r'$Q_\text{max} = 0.01$ Å$^{-1}$')
plt.legend()
plt.grid()
plt.savefig("docs/analytical-P-log.eps",bbox_inches="tight", pad_inches=0, format='eps')

In [ ]:
from definitions import *
from scipy import integrate

def form_factor(Q, R):
    Q = np.abs(Q)
    return (3 * (np.sin(Q * R) - Q * R * np.cos(Q * R))/ (Q * R) ** 3)**2
def Qy_to_y(Q_y, s_y, k):
    y = L_s * np.tan(2 * np.arcsin(Q_y / (2 * k)))
    return y + s_y

def y_to_Qy(y, s_y, k):
    Qy = 2 * k * np.sin(np.arctan((y - s_y) / L_s)/2)
    return Qy

def form_factor_y(y,s_y, k, R):
    Qy = y_to_Qy(y,s_y, k)
    return form_factor(Qy,R)
    
def ff_area(R, k):
    y_lim = 0.025e-7 / R
    y_oversized = np.linspace(-y_lim,y_lim, 100000)
    dy = y_oversized[1] - y_oversized[0]
    Q = y_to_Qy(y_oversized,0,k)
    ff = form_factor(Q,R)
    ff_area = integrate.trapezoid(ff) * dy
    # print(ff_area)
    return ff_area

R = 50e-9
Q_max = 0.015e10
y_lim = 0.025e-7 / R
y_oversized = np.linspace(-y_lim,y_lim, 100000)
# dQ = Q[1] - Q[0]
norm_map = {}
for lambda_0 in [4.321e-10, 8e-10]:
    k = 2 * np.pi / lambda_0
    L_s = 1.8
    Q = y_to_Qy(y_oversized,0,k)
    ff = form_factor(Q,R)
    # plt.plot(Q * 1e-10,ff)
    ff_norm_factor = ff_area(R, k)
    ff_norm = ff / ff_norm_factor
    # norm_map[lambda_0] = ff_area
    # print(ff_area)
    # ff_norm = ff / ff_area
    y = Qy_to_y(Q, 0, k=k)
    plt.plot(y * 1e3,ff_norm, label=rf'$\lambda_0 = {lambda_0 * 1e10}Å$')
    # plt.yscale('log')
    plt.legend()
    plt.xlabel(r'$y$ [mm]')
    plt.ylabel(r'$P(y)$')

In [ ]:
from definitions import *
from scipy import integrate

def form_factor(Q, R):
    Q = np.abs(Q)
    return (3 * (np.sin(Q * R) - Q * R * np.cos(Q * R))/ (Q * R) ** 3)**2
# Q_max = 0.015e10
# Q = np.linspace(-Q_max, Q_max,100000)
h_d = detector_size
y = np.linspace(-h_d / 2, h_d/2, 10000)
# dQ = Q[1] - Q[0]
dy = y[1] - y[0]
print(R)
for (lambda_0, delta_limit) in [(4.321e-10, 70.71e-9), (8e-10,130.91)]:
    for R in [50e-9, 100e-9, 200e-9, 2000e-9]:
        # R = 300e-9
        k = 2 * np.pi / lambda_0
        L_s = 1.8
        N_sample_points = 10000
        s_y_range = np.linspace(-0.005,0.005,N_sample_points)
        I_t = np.zeros_like(y)
        norm_factor = ff_area(R, k)
        for s_y in s_y_range:
            ff = form_factor_y(y, s_y, k, R) / (N_sample_points * norm_factor)
            # plt.plot(y, ff)
            # plt.plot(y[indices] * 1e3,ff[indices])
            I_t += ff * 0.01
            # plt.yscale('log')
        # plt.plot(y)
        plt.plot(y * 1e3, I_t, label=rf'$R = {round(R * 1e9)}$nm')
        plt.xlabel(r'$y$ [mm]')
        plt.ylabel(r'$I(y)$')
    plt.legend()
    plt.xlim((-h_d/2 * 1e3,h_d/2 * 1e3))
    plt.grid()
    plt.savefig(f"docs/simplified-I-scattering-{lambda_0 * 1e10}.eps",bbox_inches="tight", pad_inches=0, format='eps')

    plt.show()
# y  = np.linspace(-10e-3, 10e-3)


In [ ]:
h_d / (L_s *2)